## install relevant packages

In [ ]:
pip install transformers torch

## Import libraries

In [20]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

## Define Chatbot class functions

In [14]:
class Chatbot:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained("microsoft/DialoGPT-medium")
        self.model = AutoModelForCausalLM.from_pretrained("microsoft/DialoGPT-medium")
        self.chat_history_ids = None

    def reset_context(self):
        self.chat_history_ids = None

    # Modify the get_response method to include do_sample=True in the model.generate call

    def get_response(self, message):
        # Encode the new user message
        new_user_input_ids = self.tokenizer.encode(message + self.tokenizer.eos_token, return_tensors='pt')

        # Append the new user input tokens to chat history, if it exists
        bot_input_ids = torch.cat([self.chat_history_ids, new_user_input_ids], dim=-1) if self.chat_history_ids is not None else new_user_input_ids

        # Generate a response with do_sample=True to enable sample-based generation
        self.chat_history_ids = self.model.generate(
            bot_input_ids,
            max_length=1000,
            pad_token_id=self.tokenizer.eos_token_id,
            temperature=0.7,
            do_sample=True
        )

        # Extract the response
        response = self.tokenizer.decode(self.chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)

        return response

chatbot = Chatbot()



## Test the chatbot

In [ ]:
# Example conversation
print("Bot: Hi! How can I help you today?")
while True:
    message = input("You: ")
    if message.lower() == "quit":
        break
    response = chatbot.get_response(message)
    print(f'You: {message}')
    print(f"Bot: {response}")